In [1]:
import duckdb

%load_ext sql
conn = duckdb.connect("../dev.duckdb")
%sql conn --alias duckdb

In [2]:
%sql show tables;

Running query in 'duckdb'

name
returns
sales


# Escaping SQL Traps with USS
https://showmethedata.blog/sql-traps-unified-star-schema

## Raw data

In [3]:
%sql select * from sales;

Running query in 'duckdb'

sale_id,sale_date,product,qty_sold
1,2023-01-01,PR01,2
2,2023-01-02,PR02,5
3,2023-01-03,PR03,5


In [4]:
%sql select * from returns;

Running query in 'duckdb'

return_id,sale_id,return_date,qty_returned
1,1,2023-01-02,2
2,2,2023-01-03,5
3,3,2023-01-04,2
4,3,2023-01-05,3


# Fan Trap
> In data modelling, a fan trap occurs when multiple rows in one table refer to a single key in another table containing measures, leading to a duplication of the measures in the final joined result.

In this case `Returns` table might hold many refereces to a single `Sale` (many to one). This will lead to `Sales` explosion. 

In [5]:
%%sql
select
    *
from returns
join sales using (sale_id);

Running query in 'duckdb'

return_id,sale_id,return_date,qty_returned,sale_date,product,qty_sold
1,1,2023-01-02,2,2023-01-01,PR01,2
2,2,2023-01-03,5,2023-01-02,PR02,5
3,3,2023-01-04,2,2023-01-03,PR03,5
4,3,2023-01-05,3,2023-01-03,PR03,5


# Chasm Trap
> The chasm trap occurs when a target table (referred to via foreign keys) gets exploded by 2 or more other tables, causing an effect similar to a cartesian product. Since the number of participating tables is unbounded and they all explode each other, the chasm trap can produce many more unwanted duplicates than a fan trap.